# KL divergence estimation

Idea from [here](https://stats.stackexchange.com/questions/211175/kullback-leibler-divergence-for-two-samples) and [this paper](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.422.5121&rep=rep1&type=pdf).

In [ ]:
def empirical_pdf(xs):
    """
    Arguments:
    * xs: np.array of shape (n, d) consisting of n observations of an $\mathbb R^d$-valued random variable.
    
    Returns:
    * callable that takes one argument: an np.array of shape (d,) representing an $\mathbb R^d$-valued
    random variable.
    
    Empirical pdf of particles, computed by method in paper linked above.
    """
    d = xs.shape[1]
    
    
    raise NotImplementedError()

In [ ]:
# not the best way to do this
# cause then we need to integrate
# better to expectation with samples?
# OTOH, linear spline is prob a better approximation that empirical cdf
# TODO: can we make a better approximation of the KSD this way?
def kl_divergence(q, p):
    """
    Arguments:
    * q, p: callables. Both take one argument: an np.array of shape (d,) representing an $\mathbb R^d$-valued
    random variable
    
    Returns:
    * KL-divergence KL(q || p) from p to q.
    """
    raise NotImplementedError()